In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv("womens_clothing_ecommerce_reviews.csv")
print(df.head())

df.info()

X = df['Review Text']
y = df['sentiment']

#'satisfy=y' ensures that the propoption of positive and negative
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nConverting text to numericsl features using Bag-of-words...")

vectorizer = CountVectorizer(stop_words='english')

X_train_bow = vectorizer.fit_transform(X_train)

X_test_bow = vectorizer.transform(X_test)

print("Text successfully converted to feature vectors")


model = LogisticRegression(max_iter=2000)

model.fit(X_train_bow, y_train)
print("Model training complete")


y_pred = model.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f} ({accuracy:.2%})")



                                         Review Text  sentiment
0  Absolutely wonderful - silky and sexy and comf...          1
1  Love this dress!  it's sooo pretty.  i happene...          1
2  I love, love, love this jumpsuit. it's fun, fl...          1
3  This shirt is very flattering to all due to th...          1
4  I love tracy reese dresses, but this one is no...         -1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19818 entries, 0 to 19817
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review Text  19818 non-null  object
 1   sentiment    19818 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 309.8+ KB

Converting text to numericsl features using Bag-of-words...
Text successfully converted to feature vectors
Model training complete
Model Accuracy: 0.9294 (92.94%)


In [2]:
import joblib
# To save the model
joblib.dump(model, 'model.joblib')

['model.joblib']

In [6]:
from google.cloud import aiplatform

In [9]:
# Initialize with project ID
aiplatform.init(project='alien-vim-470515-t4', location='us-central1')

# Create the endpoint 
endpoint = aiplatform.Endpoint(
    endpoint_name='4200559929104269312'
)

print("Successfully created endpoint object")
print(f"Endpoint resource name: {endpoint.resource_name}")

Successfully created endpoint object
Endpoint resource name: projects/783398064017/locations/us-central1/endpoints/4200559929104269312


In [11]:
# new review, we want to classify
new_review = ["The material felt cheap and it was not I expected"]

sparse_matrix = vectorizer.transform(new_review)

numpy_array = sparse_matrix.toarray()

processed_review = numpy_array.tolist()

print(processed_review)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
# Make the prediction call
response = endpoint.predict(instances=processed_review)

print(response)
print(type(response))

Prediction(predictions=[-1], deployed_model_id='1364585189535121408', metadata=None, model_version_id='1', model_resource_name='projects/783398064017/locations/us-central1/models/2030642945362755584', explanations=None)
<class 'google.cloud.aiplatform.models.Prediction'>


In [13]:
prediction_result = response.predictions[0]
sentiment = "positive" if prediction_result == 1 else "negative"
print(f"Prediction: The review sentiment is '{sentiment}'")

Prediction: The review sentiment is 'negative'
